In [ ]:
conda install keras

In [ ]:
conda remove tensorflow

In [ ]:
import cv2
import pandas as pd
import numpy as np

In [ ]:
import os
import sys
from skimage.io import imread
import matplotlib.pyplot as plt
from PIL import Image
import urllib.request

In [ ]:
from keras.applications.imagenet_utils import decode_predictions
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D
from tensorflow.python.keras.layers import Activation, Dropout, Flatten, Dense

In [ ]:
import shutil
import os

In [ ]:
df = pd.read_csv('data.tsv', sep='\t')

In [ ]:
df = df.iloc[:,0:2]

In [ ]:
df

In [ ]:
df.rename(columns={'INPUT:image':'image', 'OUTPUT:result':'target'}, inplace=True)

In [ ]:
df

In [ ]:
df_ok = df.loc[df['target'] == 'OK']

In [ ]:
df_bad = df.loc[df['target'] == 'BAD']

In [ ]:
### Create folders with image classifier

In [ ]:
df_ok.info()

In [ ]:
for image in range(5500):
    with urllib.request.urlopen(df_bad.iloc[image, 0]) as url:
        with open('/Users/image_BAD{}.jpg'.format(image), 'wb') as f:
            f.write(url.read())

In [ ]:
image = imread('/Users/image_OK5.jpg')

plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.show()

In [ ]:
# Каталог с данными для обучения
train_dir = 'train'
# Каталог с данными для проверки
val_dir = 'val'
# Каталог с данными для тестирования
test_dir = 'test'
# Размеры изображения
img_width, img_height = 150, 150
# Размерность тензора на основе изображения для входных данных в нейронную сеть
input_shape = (img_width, img_height, 3)
# Количество эпох
epochs = 30
# Размер мини-выборки
batch_size = 16
# Количество изображений для обучения
nb_train_samples = 7600
# Количество изображений для проверки
nb_validation_samples = 1300
# Количество изображений для тестирования
nb_test_samples = 1300

In [ ]:
# Каталог с набором данных
data_dir = '/Users/'
# Каталог с данными для обучения
train_dir = 'train'
# Каталог с данными для проверки
val_dir = 'val'
# Каталог с данными для тестирования
test_dir = 'test'
# Часть набора данных для тестирования
test_data_portion = 0.12
# Часть набора данных для проверки
val_data_portion = 0.12
# Количество элементов данных в одном классе
nb_images = 4500

In [ ]:
def create_directory(dir_name):
    if os.path.exists(dir_name):
        shutil.rmtree(dir_name)
    os.makedirs(dir_name)
    os.makedirs(os.path.join(dir_name, "ok"))
    os.makedirs(os.path.join(dir_name, "bad"))

In [ ]:
create_directory(train_dir)
create_directory(val_dir)
create_directory(test_dir)

In [ ]:
def copy_images(start_index, end_index, source_dir, dest_dir):
    for i in range(start_index, end_index):
        try:
            shutil.copy2(os.path.join(source_dir, "image_OK" + str(i) + ".jpg"), 
                        os.path.join(dest_dir, "ok"))
            shutil.copy2(os.path.join(source_dir, "image_BAD" + str(i) + ".jpg"), 
                       os.path.join(dest_dir, "bad"))
        except:
            continue

In [ ]:
start_val_data_idx = int(nb_images * (1 - val_data_portion - test_data_portion))
start_test_data_idx = int(nb_images * (1 - test_data_portion))
print(start_val_data_idx)
print(start_test_data_idx)

In [ ]:
copy_images(0, start_val_data_idx, data_dir, train_dir)
copy_images(start_val_data_idx, start_test_data_idx, data_dir, val_dir)
copy_images(start_test_data_idx, nb_images, data_dir, test_dir)

In [ ]:
# Каталог с данными для обучения
train_dir = 'train'
# Каталог с данными для проверки
val_dir = 'val'
# Каталог с данными для тестирования
test_dir = 'test'
# Размеры изображения
img_width, img_height = 150, 150
# Размерность тензора на основе изображения для входных данных в нейронную сеть
input_shape = (img_width, img_height, 3)
# Количество эпох
epochs = 30
# Размер мини-выборки
batch_size = 16
# Количество изображений для обучения
nb_train_samples = 9000
# Количество изображений для проверки
nb_validation_samples = 1000
# Количество изображений для тестирования
nb_test_samples = 1000

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
datagen = ImageDataGenerator(rescale=1. / 255)

In [ ]:
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

In [ ]:
val_generator = datagen.flow_from_directory(
    val_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

In [ ]:
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=nb_validation_samples // batch_size)

In [ ]:
scores = model.evaluate_generator(test_generator, nb_test_samples // batch_size)

In [ ]:
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
model.save("cv_model.h5")

In [ ]:
model.save_weights('cv_model_weights.h5')

In [ ]:
import numpy as np
from PIL import Image
from keras.preprocessing import image
img = image.load_img('/Users/image_OK13.jpg') # , target_size=(32,32))
img  = image.img_to_array(img)
img  = img.reshape((1,) + img.shape)

In [ ]:
img = image.load_img('/Users/OK2.jpg', target_size=(150,150))
img  = image.img_to_array(img)
img  = img.reshape((1,) + img.shape)
img  = img/255
predict = model.predict_classes(img)

In [ ]:
def test_():
    from skimage.io import imread
    for i in files:
        img = image.load_img('/Users/qwe/{}'.format(i), target_size=(150,150))
        images = imread(img)
        plt.figure(figsize=(10, 10))
        plt.imshow(images)
        plt.show()
        img  = image.img_to_array(img)
        img  = img.reshape((1,) + img.shape)
        img  = img/255
        answer = model.predict_classes(img)
        if answer == 1:
            print("Весы есть")
        else:
            print("Весов нет")